In [1]:
# Show all Jupyter output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import torch
from torch import nn
from torchvision import models

import sys
sys.path.append('..')

In [2]:
from src.datasets import TextureDataset

from src.datasets import TextureDataset

texture_dir = "../data/andrew/texture"
cloth_dir = "../data/andrew/clothing"
rois_db = "../data/andrew/rois.csv"


CROP_BOUNDS = (
    (160, 672),  # HEIGHT (determined by looking at sample frames. want to get the head)
    (14, 526),  # WIDTH (calculated by (540 - 512)/2, then centered  )
)

texture_dataset = TextureDataset(
    texture_dir, rois_db, cloth_dir, crop_bounds=CROP_BOUNDS
)

In [3]:
from torch.utils.data import DataLoader

loader = DataLoader(texture_dataset, batch_size=2)

In [4]:
iterator = iter(loader)

In [5]:
x = next(iterator)

In [6]:
from src.texture_module import TextureModule


In [7]:
generator = TextureModule(texture_channels=3, dropout=0.8)


In [8]:
x[0].shape, x[1].shape, x[2].shape

(torch.Size([2, 3, 512, 512]),
 torch.Size([2, 6, 5]),
 torch.Size([2, 3, 512, 512]))

In [9]:
sys.path.append("/home/remoteuser/faster-rcnn.pytorch/lib")
from model.roi_layers import ROIAlign
from src.nets import UNetDown, UNetUp
channels=3
dropout=0.5

In [10]:
NUM_ROI = 6
class TextureModule(nn.Module):
    def __init__(self, texture_channels=3, dropout=0.5):
        super(TextureModule, self).__init__()
        self.roi_align = ROIAlign(
            output_size=(128, 128), spatial_scale=1, sampling_ratio=1
        )

        channels = texture_channels * NUM_ROI
        self.encode = UNetDown(channels, channels)

        # UNET
        self.down_1 = UNetDown(channels + texture_channels, 64, normalize=False)
        self.down_2 = UNetDown(64, 128)
        self.down_3 = UNetDown(128, 256)
        self.down_4 = UNetDown(256, 512, dropout=dropout)
        self.down_5 = UNetDown(512, 1024, dropout=dropout)
        self.down_6 = UNetDown(1024, 1024, normalize=False, dropout=dropout)
        self.up_1 = UNetUp(1024, 1024, dropout=dropout)
        self.up_2 = UNetUp(2 * 1024, 512, dropout=dropout)
        self.up_3 = UNetUp(2 * 512, 256)
        self.up_4 = UNetUp(2 * 256, 128)
        self.up_5 = UNetUp(2 * 128, 64)

        self.upsample_and_pad = nn.Sequential(
            nn.Upsample(scale_factor=2),
            nn.ZeroPad2d((1, 0, 1, 0)),
            nn.Conv2d(128, texture_channels, 4, padding=1),
            nn.Tanh(),
        )



In [11]:
self = TextureModule()

In [12]:
input_tex = x[0]
rois = x[1]
cloth = x[2]

In [13]:
rois = rois.view(-1, 5)
rois[:, 0] = rois[:, 0] - rois[0, 0]
rois.shape

torch.Size([12, 5])

In [14]:
pooled_rois = self.roi_align(input_tex, rois)

In [15]:
bs = pooled_rois.shape[0] / NUM_ROI
bs

2.0

In [16]:
pooled_rois = pooled_rois.view(int(bs), -1, pooled_rois.shape[2], pooled_rois.shape[3])
pooled_rois.shape

torch.Size([2, 18, 128, 128])

In [17]:
encoded_tex = self.encode(pooled_rois)
encoded_tex.shape

torch.Size([2, 18, 64, 64])

In [18]:
scale_factor = input_tex.shape[2] / encoded_tex.shape[2]
scale_factor

8.0

In [19]:
from torch import nn
upsampled_tex = nn.functional.interpolate(encoded_tex, scale_factor=scale_factor)
upsampled_tex.shape

torch.Size([2, 18, 512, 512])

In [20]:
cloth.shape

torch.Size([2, 3, 512, 512])

In [21]:
tex_with_cloth = torch.cat((upsampled_tex, cloth), 1)
tex_with_cloth.shape

torch.Size([2, 21, 512, 512])

In [22]:
d1 = self.down_1(tex_with_cloth)
d1.shape

torch.Size([2, 64, 256, 256])

In [23]:
d2 = self.down_2(d1)
d2.shape

torch.Size([2, 128, 128, 128])

In [24]:
d3 = self.down_3(d2)
d3.shape
d4 = self.down_4(d3)
d4.shape
d5 = self.down_5(d4)
d5.shape
d6 = self.down_6(d5)
d6.shape
u1 = self.up_1(d6, d5)
u1.shape
u2 = self.up_2(u1, d4)
u2.shape
u3 = self.up_3(u2, d3)
u3.shape
u4 = self.up_4(u3, d2)
u4.shape
u5 = self.up_5(u4, d1)
u5.shape



torch.Size([2, 256, 64, 64])

torch.Size([2, 512, 32, 32])

torch.Size([2, 1024, 16, 16])

torch.Size([2, 1024, 8, 8])

torch.Size([2, 2048, 16, 16])

torch.Size([2, 1024, 32, 32])

torch.Size([2, 512, 64, 64])

torch.Size([2, 256, 128, 128])

torch.Size([2, 128, 256, 256])

In [25]:
us = self.upsample_and_pad(u5)

/home/remoteuser/.miniconda3/envs/compile-faster-rcnn/lib/python3.7/site-packages/torch/nn/modules/upsampling.py:129: UserWarning: nn.Upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.{} is deprecated. Use nn.functional.interpolate instead.".format(self.name))


In [26]:
us.shape

torch.Size([2, 3, 512, 512])